In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
root = ET.parse('routes/Route from 2021-02-17 09 01_20210217090145.gpx').getroot()

In [ ]:
pts = root.findall('.//{http://www.topografix.com/GPX/1/1}trkpt')
for pt in pts:
    lat = float(pt.attrib['lat'])
    lon = float(pt.attrib['lon'])
    ele = float(pt.find('{http://www.topografix.com/GPX/1/1}ele').text)
    time = pt.find('{http://www.topografix.com/GPX/1/1}time').text
    print(lat, lon, ele, time)
    break